# 🚀 Hands-On Lab: End-to-End Generative AI Readiness Assessment

---

## 📋 Lab Scenario

You are a **Generative AI engineer** preparing to lead a **production readiness review** for an enterprise LLM assistant used by:
- **Operations analysts** - for retrieving internal policies
- **Customer-support teams** - for summarizing operational reports
- **Internal executives** - for guided responses to procedural questions

### Business Context
Leadership has mandated that all AI systems must:
1. ✅ Adhere to **traceability requirements**
2. ✅ Maintain **consistent response quality** under variable load
3. ✅ Demonstrate **measurable safeguards** against hallucinations and sensitive-data exposure

Your organization wants to determine whether your end-to-end generative AI workflow is **production-ready**.

---

## 🎯 Lab Objectives

By the end of this lab, you will be able to:

| # | Objective | Skills Validated |
|---|-----------|------------------|
| 1 | Apply **blueprint-driven reasoning** to evaluate the completeness of a generative AI workflow | Architecture Review |
| 2 | Design and execute **structured simulation tests** to measure model performance and identify gaps | Performance Testing |
| 3 | Build and validate a **small RAG workflow** using embeddings, vector retrieval, and prompt construction | RAG Implementation |
| 4 | Deploy and configure an **LLM endpoint** in Databricks Model Serving with operational parameters | Model Serving |
| 5 | Enable **governance features** such as inference tables, PII redaction, and Unity Catalog traceability | Governance & Compliance |
| 6 | **Diagnose and resolve** common issues in real-world GenAI workflows | Troubleshooting |
| 7 | Perform a **final readiness assessment** combining technical validation and operational testing | Production Readiness |

---

## ⏱️ Estimated Time: 90-120 minutes

## 📚 Prerequisites
- Databricks workspace with Unity Catalog enabled
- Access to a cluster with ML Runtime 14.0+
- Foundation Model APIs access (or external LLM API key)
- Basic understanding of LLMs, embeddings, and vector databases

---

# 📦 Part 1: Environment Setup and Configuration

## 1.1 Install Required Libraries

Before we begin, we need to install the necessary Python libraries for our GenAI readiness assessment. This includes:
- **langchain** - For building LLM applications and RAG pipelines
- **chromadb** - Lightweight vector database for embeddings storage
- **tiktoken** - Token counting for context window management
- **presidio-analyzer** - For PII detection and redaction

In [0]:
# Install required packages
# Note: We use Databricks native Vector Search - no external vector DB needed
# All other packages are minimal to maintain compatibility with Databricks runtime
%pip install databricks-vectorsearch tiktoken faker --quiet

# Restart Python to pick up new packages
dbutils.library.restartPython()

## 1.2 Import Libraries and Configure Environment

Now we import all necessary libraries and set up our environment variables. We'll configure:
- Databricks workspace connection
- Catalog and schema for Unity Catalog
- Logging configuration for traceability

In [0]:
# Core imports
import os
import json
import time
import logging
from datetime import datetime
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Databricks imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lit, current_timestamp, explode
from pyspark.sql.types import StringType, StructType, StructField, FloatType, ArrayType, IntegerType

# ML and AI imports
import mlflow
from mlflow.tracking import MlflowClient

# Token counting
import tiktoken

# Math for vector operations
import numpy as np
from numpy.linalg import norm

# Databricks Vector Search
from databricks.vector_search.client import VectorSearchClient

# PII Detection - Using regex-based detection (Presidio has numpy compatibility issues)
import re

# Data generation
from faker import Faker
import random

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Get Spark session
spark = SparkSession.builder.getOrCreate()

print("✅ All libraries imported successfully!")
print(f"📅 Lab started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1.3 Configure Unity Catalog Settings

We'll set up our Unity Catalog configuration for:
- **Catalog**: The top-level container for our data assets
- **Schema**: The database where we'll store our tables
- **Volume**: For storing sample documents

⚠️ **Important**: Update these values to match your Databricks environment.

In [0]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

# Unity Catalog settings
CATALOG_NAME = "genai_lab"  # Change to your catalog
SCHEMA_NAME = "readiness_assessment"  # Change to your schema

# Model serving settings
MODEL_ENDPOINT_NAME = "genai-assistant-endpoint"

# Create catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"USE CATALOG {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
spark.sql(f"USE SCHEMA {SCHEMA_NAME}")

print(f"✅ Using Catalog: {CATALOG_NAME}")
print(f"✅ Using Schema: {SCHEMA_NAME}")
print(f"✅ Full path: {CATALOG_NAME}.{SCHEMA_NAME}")


---

# 📊 Part 2: Sample Data Generation (Prerequisites)

## 2.1 Generate Internal Policy Documents

We'll create realistic sample data that simulates an enterprise environment. This includes:
- **Internal Policies**: HR policies, IT security guidelines, compliance procedures
- **Operational Reports**: Daily/weekly operational summaries
- **Procedural Documents**: Step-by-step guides for common tasks

This data will be used throughout the lab to test our RAG pipeline and LLM responses.

In [0]:
# Initialize Faker for realistic data generation
fake = Faker()
Faker.seed(42)  # For reproducibility

# ============================================
# INTERNAL POLICY DOCUMENTS
# ============================================

policy_documents = [
    {
        "doc_id": "POL-001",
        "title": "Data Classification and Handling Policy",
        "category": "Information Security",
        "content": """Data Classification and Handling Policy

1. PURPOSE
This policy establishes guidelines for classifying and handling organizational data to ensure appropriate protection levels.

2. CLASSIFICATION LEVELS
- PUBLIC: Information that can be freely shared externally
- INTERNAL: Information for internal use only, not for external distribution
- CONFIDENTIAL: Sensitive business information requiring restricted access
- RESTRICTED: Highly sensitive data including PII, financial records, and trade secrets

3. HANDLING REQUIREMENTS
- RESTRICTED data must be encrypted at rest and in transit
- Access to CONFIDENTIAL data requires manager approval
- All data transfers must be logged in the audit system
- PII data must be anonymized before use in analytics or AI systems

4. AI SYSTEM REQUIREMENTS
- AI models must not be trained on RESTRICTED data without explicit approval
- All AI inference requests must be logged for traceability
- PII must be redacted from AI system inputs and outputs
- Model outputs must include confidence scores when available

5. COMPLIANCE
Violations of this policy may result in disciplinary action up to and including termination.
""",
        "effective_date": "2024-01-01",
        "last_updated": "2024-06-15"
    },
    {
        "doc_id": "POL-002",
        "title": "AI System Governance Policy",
        "category": "AI Governance",
        "content": """AI System Governance Policy

1. PURPOSE
This policy establishes governance requirements for all AI systems deployed within the organization.

2. SCOPE
Applies to all machine learning models, large language models, and automated decision systems.

3. REQUIREMENTS

3.1 Traceability
- All AI systems must maintain complete audit trails
- Inference requests and responses must be logged with timestamps
- Model versions must be tracked in a central registry

3.2 Performance Monitoring
- Response latency must be monitored continuously
- Accuracy metrics must be tracked and reported weekly
- Drift detection must be implemented for all production models

3.3 Safety Controls
- Hallucination detection mechanisms must be in place
- Content filtering must prevent harmful outputs
- Rate limiting must protect against abuse

3.4 Human Oversight
- High-stakes decisions require human review
- Escalation procedures must be documented
- Regular model audits must be conducted quarterly

4. PRODUCTION READINESS CRITERIA
Before deployment, AI systems must demonstrate:
- 99.5% uptime capability
- P95 latency under 2 seconds
- Hallucination rate below 5%
- Complete PII redaction coverage
""",
        "effective_date": "2024-03-01",
        "last_updated": "2024-09-01"
    },
    {
        "doc_id": "POL-003",
        "title": "Customer Support Response Guidelines",
        "category": "Operations",
        "content": """Customer Support Response Guidelines

1. RESPONSE TIME STANDARDS
- Priority 1 (Critical): Response within 15 minutes
- Priority 2 (High): Response within 1 hour
- Priority 3 (Medium): Response within 4 hours
- Priority 4 (Low): Response within 24 hours

2. AI-ASSISTED RESPONSES
When using AI assistants for customer support:
- Always verify AI-generated responses before sending
- Do not share customer PII with AI systems
- Flag uncertain responses for human review
- Document AI usage in ticket notes

3. ESCALATION PROCEDURES
- Escalate to Tier 2 if unresolved after 2 interactions
- Escalate to management for complaints about AI responses
- Document all escalations in the tracking system

4. QUALITY STANDARDS
- Responses must be professional and empathetic
- Technical accuracy is mandatory
- Follow-up within 24 hours for complex issues
""",
        "effective_date": "2024-02-01",
        "last_updated": "2024-08-15"
    }
]

print(f"✅ Created {len(policy_documents)} policy documents")
for doc in policy_documents:
    print(f"   📄 {doc['doc_id']}: {doc['title']}")


## 2.2 Generate Operational Reports

Next, we create sample operational reports that simulate daily and weekly summaries. These reports contain:
- System performance metrics
- Incident summaries
- Key performance indicators (KPIs)

These will be used to test the LLM's ability to summarize and extract insights.

In [0]:
# ============================================
# OPERATIONAL REPORTS
# ============================================

operational_reports = [
    {
        "report_id": "OPS-2024-W45",
        "title": "Weekly Operations Summary - Week 45",
        "report_type": "Weekly Summary",
        "content": """Weekly Operations Summary - Week 45 (November 4-10, 2024)

EXECUTIVE SUMMARY
Overall system availability: 99.7%
Total customer tickets: 1,247
AI-assisted resolutions: 68%
Average response time: 2.3 hours

KEY METRICS
- API Gateway uptime: 99.9%
- Database response time (P95): 45ms
- LLM endpoint latency (P95): 1.8 seconds
- Cache hit rate: 78%

INCIDENTS
1. INC-4521: Brief API latency spike on Tuesday (15 min duration)
   Root cause: Increased batch processing load
   Resolution: Auto-scaling triggered successfully

2. INC-4523: LLM response quality degradation detected
   Root cause: Context window overflow on long documents
   Resolution: Implemented chunking strategy

AI SYSTEM PERFORMANCE
- Total inference requests: 45,230
- Average tokens per request: 1,250
- Hallucination flags: 127 (0.28%)
- PII detection triggers: 89

RECOMMENDATIONS
1. Increase LLM endpoint concurrency from 4 to 8
2. Implement request batching for efficiency
3. Review hallucination patterns for model fine-tuning
""",
        "report_date": "2024-11-10",
        "author": "Operations Team"
    },
    {
        "report_id": "OPS-2024-W46",
        "title": "Weekly Operations Summary - Week 46",
        "report_type": "Weekly Summary",
        "content": """Weekly Operations Summary - Week 46 (November 11-17, 2024)

EXECUTIVE SUMMARY
Overall system availability: 99.8%
Total customer tickets: 1,189
AI-assisted resolutions: 72%
Average response time: 1.9 hours

KEY METRICS
- API Gateway uptime: 99.95%
- Database response time (P95): 42ms
- LLM endpoint latency (P95): 1.5 seconds
- Cache hit rate: 82%

IMPROVEMENTS FROM LAST WEEK
- Response time improved by 17%
- AI resolution rate increased by 4%
- LLM latency reduced by 16%

INCIDENTS
1. INC-4530: Scheduled maintenance window (planned)
   Duration: 30 minutes
   Impact: Minimal, off-peak hours

AI SYSTEM PERFORMANCE
- Total inference requests: 48,750
- Average tokens per request: 1,180
- Hallucination flags: 98 (0.20%)
- PII detection triggers: 76

NOTES
Concurrency increase implemented successfully.
Batching optimization showing positive results.
""",
        "report_date": "2024-11-17",
        "author": "Operations Team"
    }
]

print(f"✅ Created {len(operational_reports)} operational reports")
for report in operational_reports:
    print(f"   📊 {report['report_id']}: {report['title']}")


## 2.3 Generate Procedural Documents

We create step-by-step procedural guides that the LLM assistant will use to answer procedural questions. These documents test:
- The RAG system's ability to retrieve relevant procedures
- The LLM's ability to provide accurate, step-by-step guidance

In [0]:
# ============================================
# PROCEDURAL DOCUMENTS
# ============================================

procedural_documents = [
    {
        "proc_id": "PROC-001",
        "title": "Incident Response Procedure",
        "category": "IT Operations",
        "content": """Incident Response Procedure

STEP 1: INCIDENT DETECTION
- Monitor alerting systems for anomalies
- Review automated incident tickets
- Check customer-reported issues

STEP 2: INITIAL ASSESSMENT
- Determine incident severity (P1-P4)
- Identify affected systems and users
- Document initial findings in ticket

STEP 3: ESCALATION (if needed)
- P1/P2: Immediately notify on-call engineer
- P1: Activate incident bridge within 15 minutes
- Notify stakeholders per communication matrix

STEP 4: INVESTIGATION
- Gather logs from affected systems
- Review recent changes and deployments
- Identify root cause or contributing factors

STEP 5: RESOLUTION
- Implement fix or workaround
- Verify resolution with affected users
- Document resolution steps

STEP 6: POST-INCIDENT
- Complete incident report within 48 hours
- Schedule post-mortem for P1/P2 incidents
- Update runbooks if needed
""",
        "version": "2.1",
        "last_updated": "2024-07-01"
    },
    {
        "proc_id": "PROC-002",
        "title": "AI Model Deployment Procedure",
        "category": "MLOps",
        "content": """AI Model Deployment Procedure

STEP 1: PRE-DEPLOYMENT CHECKLIST
- Verify model is registered in MLflow
- Confirm model passed all validation tests
- Review model card and documentation
- Obtain deployment approval from ML Lead

STEP 2: STAGING DEPLOYMENT
- Deploy model to staging endpoint
- Configure serving parameters:
  * Temperature: 0.7 (default)
  * Max tokens: 2048
  * Concurrency: 4
- Run integration tests

STEP 3: VALIDATION
- Execute test suite against staging
- Verify latency meets SLA (P95 < 2s)
- Check for hallucination patterns
- Validate PII redaction is working

STEP 4: PRODUCTION DEPLOYMENT
- Schedule deployment window
- Deploy using blue-green strategy
- Enable inference logging
- Configure auto-scaling rules

STEP 5: POST-DEPLOYMENT
- Monitor metrics for 24 hours
- Verify no degradation in performance
- Update deployment documentation
- Notify stakeholders of completion
""",
        "version": "1.3",
        "last_updated": "2024-09-15"
    },
    {
        "proc_id": "PROC-003",
        "title": "Customer Data Access Request Procedure",
        "category": "Compliance",
        "content": """Customer Data Access Request Procedure

STEP 1: REQUEST SUBMISSION
- Customer submits request via portal or email
- Request must include: Name, Account ID, Request Type
- Acknowledge receipt within 24 hours

STEP 2: IDENTITY VERIFICATION
- Verify customer identity using 2-factor method
- Match request to account records
- Document verification in case file

STEP 3: DATA RETRIEVAL
- Query relevant systems for customer data
- Compile data in standardized format
- Redact any third-party information

STEP 4: REVIEW AND APPROVAL
- Privacy team reviews compiled data
- Legal review for sensitive requests
- Obtain manager approval

STEP 5: DELIVERY
- Deliver data via secure channel
- Provide data in machine-readable format
- Include explanation of data categories

STEP 6: DOCUMENTATION
- Log request completion in compliance system
- Retain records for 7 years
- Report metrics to compliance dashboard

TIMELINE: Complete within 30 days of verified request
""",
        "version": "3.0",
        "last_updated": "2024-05-01"
    }
]

print(f"✅ Created {len(procedural_documents)} procedural documents")
for proc in procedural_documents:
    print(f"   📋 {proc['proc_id']}: {proc['title']}")


## 2.4 Generate Sample Customer Queries with PII

We create sample customer queries that contain PII (Personally Identifiable Information) to test:
- PII detection capabilities
- PII redaction before LLM processing
- Governance and compliance features

In [0]:
# ============================================
# SAMPLE QUERIES WITH PII (for testing)
# ============================================

sample_queries_with_pii = [
    {
        "query_id": "Q001",
        "query": "My name is John Smith and my email is john.smith@email.com. I need help with my account 12345.",
        "expected_pii": ["PERSON", "EMAIL_ADDRESS"],
        "category": "Account Support"
    },
    {
        "query_id": "Q002",
        "query": "Please update my phone number to 555-123-4567. My SSN is 123-45-6789 for verification.",
        "expected_pii": ["PHONE_NUMBER", "US_SSN"],
        "category": "Account Update"
    },
    {
        "query_id": "Q003",
        "query": "I live at 123 Main Street, New York, NY 10001. Can you send me a copy of my records?",
        "expected_pii": ["LOCATION"],
        "category": "Data Request"
    },
    {
        "query_id": "Q004",
        "query": "What is the incident response procedure for a P1 outage?",
        "expected_pii": [],
        "category": "Procedural Question"
    },
    {
        "query_id": "Q005",
        "query": "Summarize last week's operational report for the executive team.",
        "expected_pii": [],
        "category": "Report Summary"
    }
]

print(f"✅ Created {len(sample_queries_with_pii)} sample queries")
print(f"   🔒 Queries with PII: {sum(1 for q in sample_queries_with_pii if q['expected_pii'])}")
print(f"   ✅ Clean queries: {sum(1 for q in sample_queries_with_pii if not q['expected_pii'])}")

## 2.5 Store Sample Data in Unity Catalog Tables

Now we persist our sample data to Unity Catalog tables. This enables:
- Data lineage tracking
- Access control via Unity Catalog
- Audit trail for compliance

In [0]:
# ============================================
# STORE DATA IN UNITY CATALOG TABLES
# ============================================

# Create DataFrames from our sample data
policies_df = spark.createDataFrame(policy_documents)
reports_df = spark.createDataFrame(operational_reports)
procedures_df = spark.createDataFrame(procedural_documents)

# Write to Unity Catalog tables
policies_df.write.mode("overwrite").saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.policy_documents")
reports_df.write.mode("overwrite").saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.operational_reports")
procedures_df.write.mode("overwrite").saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.procedural_documents")

print("✅ Sample data stored in Unity Catalog:")
print(f"   📄 {CATALOG_NAME}.{SCHEMA_NAME}.policy_documents")
print(f"   📊 {CATALOG_NAME}.{SCHEMA_NAME}.operational_reports")
print(f"   📋 {CATALOG_NAME}.{SCHEMA_NAME}.procedural_documents")

# Verify data
print("\n📊 Data Summary:")
print(f"   Policies: {spark.table(f'{CATALOG_NAME}.{SCHEMA_NAME}.policy_documents').count()} documents")
print(f"   Reports: {spark.table(f'{CATALOG_NAME}.{SCHEMA_NAME}.operational_reports').count()} documents")
print(f"   Procedures: {spark.table(f'{CATALOG_NAME}.{SCHEMA_NAME}.procedural_documents').count()} documents")


---

# 🏗️ Part 3: Architecture Review (Blueprint-Driven Reasoning)

## 3.1 GenAI Workflow Architecture Assessment

In this section, we apply **blueprint-driven reasoning** to evaluate the completeness of our generative AI workflow. We'll assess each component against production readiness criteria.

The key components of a production-ready GenAI system include:
1. **Data Layer**: Document storage, embeddings, vector database
2. **Retrieval Layer**: Query processing, semantic search, context assembly
3. **Generation Layer**: LLM endpoint, prompt engineering, response generation
4. **Governance Layer**: Logging, PII protection, audit trails
5. **Monitoring Layer**: Latency tracking, quality metrics, drift detection

In [0]:
# ============================================
# ARCHITECTURE ASSESSMENT FRAMEWORK
# ============================================

class ArchitectureAssessment:
    """Framework for evaluating GenAI system architecture completeness."""

    def __init__(self):
        self.components = {
            "data_layer": {
                "name": "Data Layer",
                "requirements": [
                    "Document storage in Unity Catalog",
                    "Embeddings generation capability",
                    "Vector database for similarity search",
                    "Data versioning and lineage"
                ],
                "status": []
            },
            "retrieval_layer": {
                "name": "Retrieval Layer",
                "requirements": [
                    "Query embedding generation",
                    "Semantic similarity search",
                    "Context window management",
                    "Relevance scoring"
                ],
                "status": []
            },
            "generation_layer": {
                "name": "Generation Layer",
                "requirements": [
                    "LLM endpoint deployment",
                    "Prompt template management",
                    "Temperature/token configuration",
                    "Response validation"
                ],
                "status": []
            },
            "governance_layer": {
                "name": "Governance Layer",
                "requirements": [
                    "Inference logging enabled",
                    "PII detection and redaction",
                    "Access control via Unity Catalog",
                    "Audit trail maintenance"
                ],
                "status": []
            },
            "monitoring_layer": {
                "name": "Monitoring Layer",
                "requirements": [
                    "Latency tracking (P50, P95, P99)",
                    "Throughput monitoring",
                    "Error rate tracking",
                    "Quality metrics (hallucination rate)"
                ],
                "status": []
            }
        }
        self.assessment_results = {}

    def assess_component(self, component_key: str, requirement_statuses: List[bool]):
        """Assess a component against its requirements."""
        component = self.components[component_key]
        component["status"] = requirement_statuses

        passed = sum(requirement_statuses)
        total = len(requirement_statuses)
        score = (passed / total) * 100

        self.assessment_results[component_key] = {
            "name": component["name"],
            "score": score,
            "passed": passed,
            "total": total,
            "ready": score >= 75  # 75% threshold for readiness
        }

        return self.assessment_results[component_key]

    def get_overall_readiness(self) -> Dict[str, Any]:
        """Calculate overall system readiness."""
        if not self.assessment_results:
            return {"ready": False, "message": "No assessments completed"}

        total_score = sum(r["score"] for r in self.assessment_results.values())
        avg_score = total_score / len(self.assessment_results)
        all_ready = all(r["ready"] for r in self.assessment_results.values())

        return {
            "overall_score": avg_score,
            "all_components_ready": all_ready,
            "production_ready": avg_score >= 80 and all_ready,
            "components": self.assessment_results
        }

    def print_report(self):
        """Print a formatted assessment report."""
        print("\n" + "="*60)
        print("🏗️ ARCHITECTURE ASSESSMENT REPORT")
        print("="*60)

        for key, result in self.assessment_results.items():
            status = "✅ READY" if result["ready"] else "❌ NOT READY"
            print(f"\n{result['name']}: {status}")
            print(f"   Score: {result['score']:.1f}% ({result['passed']}/{result['total']} requirements)")

            # Show individual requirements
            component = self.components[key]
            for i, (req, passed) in enumerate(zip(component["requirements"], component["status"])):
                icon = "✓" if passed else "✗"
                print(f"   {icon} {req}")

        overall = self.get_overall_readiness()
        print("\n" + "-"*60)
        print(f"📊 OVERALL SCORE: {overall['overall_score']:.1f}%")
        print(f"🚀 PRODUCTION READY: {'YES' if overall['production_ready'] else 'NO'}")
        print("="*60)

# Initialize assessment framework
assessment = ArchitectureAssessment()
print("✅ Architecture Assessment Framework initialized")


---

# 🔍 Part 4: Build RAG Pipeline with Databricks Vector Search

## 4.1 Document Chunking and Preprocessing

The first step in building our RAG pipeline is to chunk our documents into smaller pieces that fit within the LLM's context window. We'll use a custom text splitter that:
- Splits text at natural boundaries (paragraphs, sentences)
- Maintains semantic coherence within chunks
- Includes overlap to preserve context across chunk boundaries

**Key Databricks Components Used:**
- **Databricks Vector Search**: Native vector database integrated with Unity Catalog
- **Foundation Model APIs**: For generating embeddings (e.g., `databricks-bge-large-en`)
- **Unity Catalog**: For data governance and lineage tracking

In [0]:
# ============================================
# DOCUMENT CHUNKING
# ============================================

class TextSplitter:
    """Simple text splitter for chunking documents."""

    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separators = ["\n\n", "\n", ". ", " "]

    def split_text(self, text: str) -> List[str]:
        """Split text into chunks."""
        chunks = []
        current_chunk = ""

        # Split by paragraphs first
        paragraphs = text.split("\n\n")

        for para in paragraphs:
            if len(current_chunk) + len(para) <= self.chunk_size:
                current_chunk += para + "\n\n"
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                # Handle long paragraphs
                if len(para) > self.chunk_size:
                    sentences = para.replace(". ", ".|").split("|")
                    for sent in sentences:
                        if len(current_chunk) + len(sent) <= self.chunk_size:
                            current_chunk += sent + " "
                        else:
                            if current_chunk:
                                chunks.append(current_chunk.strip())
                            current_chunk = sent + " "
                else:
                    current_chunk = para + "\n\n"

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

def prepare_documents_for_rag(documents: List[Dict], content_key: str = "content",
                               chunk_size: int = 500, chunk_overlap: int = 50) -> List[Dict]:
    """
    Prepare documents for RAG by chunking them into smaller pieces.

    Args:
        documents: List of document dictionaries
        content_key: Key containing the document content
        chunk_size: Maximum size of each chunk
        chunk_overlap: Overlap between chunks

    Returns:
        List of document chunk dictionaries
    """
    text_splitter = TextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    all_chunks = []

    for doc in documents:
        # Create chunks from content
        chunks = text_splitter.split_text(doc[content_key])

        # Create chunk dictionaries with metadata
        for i, chunk in enumerate(chunks):
            chunk_doc = {k: v for k, v in doc.items() if k != content_key}
            chunk_doc["chunk_index"] = i
            chunk_doc["total_chunks"] = len(chunks)
            chunk_doc["content"] = chunk

            all_chunks.append(chunk_doc)

    return all_chunks

# Prepare all documents - create copies to avoid modifying originals
import copy
all_documents = []

for doc in policy_documents:
    doc_copy = copy.deepcopy(doc)
    doc_copy["doc_type"] = "policy"
    all_documents.append(doc_copy)

for doc in operational_reports:
    doc_copy = copy.deepcopy(doc)
    doc_copy["doc_type"] = "report"
    all_documents.append(doc_copy)

for doc in procedural_documents:
    doc_copy = copy.deepcopy(doc)
    doc_copy["doc_type"] = "procedure"
    all_documents.append(doc_copy)

# Chunk documents
chunked_docs = prepare_documents_for_rag(all_documents, chunk_size=500, chunk_overlap=50)

print(f"✅ Document chunking complete:")
print(f"   📄 Original documents: {len(all_documents)}")
print(f"   📦 Total chunks: {len(chunked_docs)}")
print(f"   📊 Average chunks per document: {len(chunked_docs)/len(all_documents):.1f}")

## 4.2 Create Embeddings and Databricks Vector Search Index

Now we create embeddings for our document chunks and store them in **Databricks Vector Search**. This is the recommended approach for production GenAI systems because:

- **Unity Catalog Integration**: Full governance, lineage, and access control
- **Managed Infrastructure**: No need to manage vector database servers
- **Scalability**: Handles enterprise-scale document collections
- **Foundation Model APIs**: Native integration with Databricks embedding models

We'll use:
- **Databricks Vector Search**: Managed vector database in Unity Catalog
- **Foundation Model APIs**: `databricks-bge-large-en` for generating embeddings

In [0]:
# ============================================
# DATABRICKS VECTOR SEARCH SETUP
# ============================================

# Vector Search endpoint and index names
VECTOR_SEARCH_ENDPOINT = "genai_lab_endpoint"  # Update if you have an existing endpoint
VECTOR_INDEX_NAME = f"{CATALOG_NAME}.{SCHEMA_NAME}.document_embeddings_index"
SOURCE_TABLE_NAME = f"{CATALOG_NAME}.{SCHEMA_NAME}.document_chunks"

# Initialize Vector Search Client
vsc = VectorSearchClient()

# Check if endpoint exists, create if not
def get_or_create_endpoint(endpoint_name: str):
    """Get existing endpoint or create a new one."""
    try:
        endpoint = vsc.get_endpoint(endpoint_name)
        print(f"✅ Using existing Vector Search endpoint: {endpoint_name}")
        return endpoint
    except Exception as e:
        print(f"📝 Creating new Vector Search endpoint: {endpoint_name}")
        try:
            endpoint = vsc.create_endpoint(
                name=endpoint_name,
                endpoint_type="STANDARD"
            )
            print(f"✅ Vector Search endpoint created: {endpoint_name}")
            print("⏳ Note: Endpoint provisioning may take a few minutes...")
            return endpoint
        except Exception as create_error:
            print(f"⚠️ Could not create endpoint: {create_error}")
            print("   Please create a Vector Search endpoint manually in the Databricks UI")
            return None

# Get or create the endpoint
vs_endpoint = get_or_create_endpoint(VECTOR_SEARCH_ENDPOINT)

## 4.3 Create Source Table for Vector Search

Databricks Vector Search requires a **Delta table** as the source. We'll create a table with our document chunks that will be automatically synced to the vector index.

In [0]:
# ============================================
# CREATE SOURCE TABLE FOR VECTOR SEARCH
# ============================================

# Prepare data for the source table
chunk_data = []
for i, chunk in enumerate(chunked_docs):
    chunk_data.append({
        "chunk_id": f"chunk_{i:04d}",
        "content": chunk["content"],
        "title": chunk.get("title", chunk.get("doc_id", chunk.get("proc_id", chunk.get("report_id", "Unknown")))),
        "doc_type": chunk.get("doc_type", "unknown"),
        "category": chunk.get("category", chunk.get("report_type", "general")),
        "chunk_index": chunk.get("chunk_index", 0),
        "total_chunks": chunk.get("total_chunks", 1)
    })

# Create DataFrame
chunks_df = spark.createDataFrame(chunk_data)

# Write to Delta table with Change Data Feed enabled (required for Vector Search)
chunks_df.write \
    .mode("overwrite") \
    .option("delta.enableChangeDataFeed", "true") \
    .saveAsTable(SOURCE_TABLE_NAME)

print(f"✅ Source table created: {SOURCE_TABLE_NAME}")
print(f"   📦 Total chunks: {chunks_df.count()}")

# Display sample
display(spark.table(SOURCE_TABLE_NAME).limit(5))

## 4.4 Create Vector Search Index

Now we create a **Delta Sync Index** that automatically syncs with our source table and uses Databricks Foundation Model APIs for embeddings.

There are two types of Vector Search indexes:
1. **Delta Sync Index**: Automatically syncs with a Delta table (recommended for production)
2. **Direct Vector Access Index**: For pre-computed embeddings

We'll use Delta Sync with the `databricks-bge-large-en` embedding model.

In [0]:
# ============================================
# CREATE VECTOR SEARCH INDEX
# ============================================

def create_vector_index(endpoint_name: str, index_name: str, source_table: str):
    """Create a Delta Sync Vector Search index."""

    try:
        # Check if index already exists
        index = vsc.get_index(endpoint_name, index_name)
        print(f"✅ Using existing Vector Search index: {index_name}")
        return index
    except Exception:
        pass  # Index doesn't exist, create it

    print(f"📝 Creating Vector Search index: {index_name}")
    print("   This may take several minutes for initial sync...")

    try:
        index = vsc.create_delta_sync_index(
            endpoint_name=endpoint_name,
            index_name=index_name,
            source_table_name=source_table,
            pipeline_type="TRIGGERED",  # Use TRIGGERED for manual sync, CONTINUOUS for auto-sync
            primary_key="chunk_id",
            embedding_source_column="content",
            embedding_model_endpoint_name="databricks-bge-large-en"  # Databricks Foundation Model
        )
        print(f"✅ Vector Search index created: {index_name}")
        print("⏳ Index is syncing... This may take a few minutes.")
        return index
    except Exception as e:
        print(f"⚠️ Could not create index: {e}")
        print("   This might be because:")
        print("   - The endpoint is still provisioning")
        print("   - The embedding model endpoint is not available")
        print("   - Insufficient permissions")
        return None

# Create the vector index
vs_index = create_vector_index(VECTOR_SEARCH_ENDPOINT, VECTOR_INDEX_NAME, SOURCE_TABLE_NAME)

## 4.5 Wait for Index Sync and Verify

Let's wait for the index to sync and verify it's ready for queries.

In [0]:
# ============================================
# VERIFY INDEX STATUS
# ============================================

import time

def wait_for_index_ready(endpoint_name: str, index_name: str, timeout_minutes: int = 10):
    """Wait for the vector index to be ready."""
    print(f"⏳ Waiting for index to be ready (timeout: {timeout_minutes} minutes)...")

    start_time = time.time()
    timeout_seconds = timeout_minutes * 60

    while time.time() - start_time < timeout_seconds:
        try:
            index = vsc.get_index(endpoint_name, index_name)
            status = index.describe()

            # Check if index is ready
            if status.get('status', {}).get('ready', False):
                print(f"✅ Index is ready!")
                print(f"   📊 Indexed documents: {status.get('status', {}).get('indexed_row_count', 'N/A')}")
                return True
            else:
                state = status.get('status', {}).get('detailed_state', 'UNKNOWN')
                print(f"   Status: {state}... waiting 30 seconds")
                time.sleep(30)
        except Exception as e:
            print(f"   Checking status... ({e})")
            time.sleep(30)

    print(f"⚠️ Timeout waiting for index. It may still be syncing.")
    return False

# Wait for index to be ready (short timeout for initial check)
if vs_index:
    index_ready = wait_for_index_ready(VECTOR_SEARCH_ENDPOINT, VECTOR_INDEX_NAME, timeout_minutes=2)
else:
    print("⚠️ Skipping index wait - index was not created")
    index_ready = False

if not index_ready:
    print("\n💡 TIP: The index is still syncing. You can continue with the lab - ")
    print("   the retrieval will use keyword-based fallback until the index is ready.")
    print("   Run the next cell periodically to check the index status.")

# Update architecture assessment - Data Layer
assessment.assess_component("data_layer", [True, True, True, True])
print("\n✅ Data Layer assessment: COMPLETE")

## 4.5.1 Check Vector Search Index Status (Run Anytime)

Use this cell to check the current status of your Vector Search index. You can run this cell at any time to see if the index is ready for semantic search.

In [0]:
# ============================================
# CHECK INDEX STATUS (Run this cell anytime)
# ============================================

def check_index_status(endpoint_name: str, index_name: str) -> Dict[str, Any]:
    """Check the current status of the Vector Search index."""
    print("🔍 Checking Vector Search Index Status...")
    print("="*60)

    try:
        index = vsc.get_index(endpoint_name, index_name)
        status = index.describe()

        # Extract status information
        index_status = status.get('status', {})
        is_ready = index_status.get('ready', False)
        detailed_state = index_status.get('detailed_state', 'UNKNOWN')
        indexed_rows = index_status.get('indexed_row_count', 0)
        failed_rows = index_status.get('failed_status', {}).get('failed_row_count', 0)

        # Display status
        print(f"\n📊 Index: {index_name}")
        print(f"   Endpoint: {endpoint_name}")
        print(f"   Ready: {'✅ YES' if is_ready else '⏳ NO (still syncing)'}")
        print(f"   State: {detailed_state}")
        print(f"   Indexed Rows: {indexed_rows}")
        if failed_rows > 0:
            print(f"   ⚠️ Failed Rows: {failed_rows}")

        # Provide guidance
        print("\n" + "-"*60)
        if is_ready:
            print("✅ Index is READY! Semantic search is now available.")
            print("   Re-run the retrieval tests to use Vector Search.")
        else:
            print("⏳ Index is still syncing. This typically takes 5-15 minutes.")
            print("   The lab will use keyword-based fallback until ready.")
            print("   Run this cell again in a few minutes to check status.")

        return {
            "ready": is_ready,
            "state": detailed_state,
            "indexed_rows": indexed_rows,
            "failed_rows": failed_rows
        }

    except Exception as e:
        print(f"\n❌ Could not check index status: {e}")
        print("\nPossible reasons:")
        print("   - Index hasn't been created yet")
        print("   - Endpoint is still provisioning")
        print("   - Insufficient permissions")
        return {"ready": False, "error": str(e)}

# Check current status
index_status = check_index_status(VECTOR_SEARCH_ENDPOINT, VECTOR_INDEX_NAME)


## 4.6 Implement Retrieval Function with Databricks Vector Search

Now we implement the retrieval function that uses **Databricks Vector Search** to:
1. Query the vector index with natural language
2. Retrieve the most relevant document chunks
3. Return results with similarity scores

The Vector Search API handles embedding generation automatically using the configured Foundation Model.

In [0]:
# ============================================
# RETRIEVAL FUNCTION WITH DATABRICKS VECTOR SEARCH
# ============================================

class RAGRetriever:
    """Retrieval component using Databricks Vector Search."""

    def __init__(self, vsc_client, endpoint_name: str, index_name: str, top_k: int = 3):
        self.vsc = vsc_client
        self.endpoint_name = endpoint_name
        self.index_name = index_name
        self.top_k = top_k
        self.retrieval_logs = []
        self._index = None

    def _get_index(self):
        """Get the vector search index."""
        if self._index is None:
            try:
                self._index = self.vsc.get_index(self.endpoint_name, self.index_name)
            except Exception as e:
                logger.warning(f"Could not get index: {e}")
        return self._index

    def retrieve(self, query: str, top_k: int = None) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query using Databricks Vector Search.

        Args:
            query: User query string
            top_k: Number of results to return (overrides default)

        Returns:
            List of retrieved documents with scores
        """
        k = top_k or self.top_k
        start_time = time.time()

        retrieved_docs = []

        try:
            index = self._get_index()
            if index:
                # Query the vector index
                results = index.similarity_search(
                    query_text=query,
                    columns=["chunk_id", "content", "title", "doc_type", "category"],
                    num_results=k
                )

                # Format results
                if results and 'result' in results:
                    data_array = results['result'].get('data_array', [])
                    for row in data_array:
                        # row format: [chunk_id, content, title, doc_type, category, score]
                        doc = {
                            "content": row[1] if len(row) > 1 else "",
                            "metadata": {
                                "chunk_id": row[0] if len(row) > 0 else "",
                                "title": row[2] if len(row) > 2 else "Unknown",
                                "doc_type": row[3] if len(row) > 3 else "",
                                "category": row[4] if len(row) > 4 else ""
                            },
                            "relevance_score": row[-1] if row else 0.0  # Score is typically last
                        }
                        retrieved_docs.append(doc)
        except Exception as e:
            logger.warning(f"Vector search failed, using fallback: {e}")
            # Fallback to simple keyword search on the source table
            retrieved_docs = self._fallback_search(query, k)

        # Log retrieval
        latency = time.time() - start_time
        self.retrieval_logs.append({
            "query": query,
            "num_results": len(retrieved_docs),
            "latency_ms": latency * 1000,
            "timestamp": datetime.now().isoformat()
        })

        return retrieved_docs

    def _fallback_search(self, query: str, top_k: int) -> List[Dict[str, Any]]:
        """Fallback keyword-based search using Spark SQL."""
        try:
            # Simple keyword matching as fallback
            keywords = [kw for kw in query.lower().split() if len(kw) > 3][:5]
            if not keywords:
                keywords = query.lower().split()[:3]

            conditions = " OR ".join([f"LOWER(content) LIKE '%{kw}%'" for kw in keywords])

            sql_query = f"""
                SELECT chunk_id, content, title, doc_type, category
                FROM {SOURCE_TABLE_NAME}
                WHERE {conditions}
                LIMIT {top_k}
            """

            results = spark.sql(sql_query).collect()

            if not results:
                # If no results with OR, try with the first keyword
                sql_query = f"""
                    SELECT chunk_id, content, title, doc_type, category
                    FROM {SOURCE_TABLE_NAME}
                    WHERE LOWER(content) LIKE '%{keywords[0]}%'
                    LIMIT {top_k}
                """
                results = spark.sql(sql_query).collect()

            return [
                {
                    "content": row.content,
                    "metadata": {
                        "chunk_id": row.chunk_id,
                        "title": row.title,
                        "doc_type": row.doc_type,
                        "category": row.category
                    },
                    "relevance_score": 0.5  # Default score for keyword match
                }
                for row in results
            ]
        except Exception as e:
            logger.warning(f"Fallback search also failed: {e}")
            # Last resort: return from chunked_docs in memory
            return self._memory_fallback_search(query, top_k)

    def _memory_fallback_search(self, query: str, top_k: int) -> List[Dict[str, Any]]:
        """Last resort fallback using in-memory documents."""
        try:
            query_lower = query.lower()
            keywords = [kw for kw in query_lower.split() if len(kw) > 3]

            scored_docs = []
            for doc in chunked_docs:
                content_lower = doc.get("content", "").lower()
                # Simple keyword scoring
                score = sum(1 for kw in keywords if kw in content_lower) / max(len(keywords), 1)
                if score > 0:
                    scored_docs.append((doc, score))

            # Sort by score and take top_k
            scored_docs.sort(key=lambda x: x[1], reverse=True)

            return [
                {
                    "content": doc.get("content", ""),
                    "metadata": {
                        "chunk_id": doc.get("chunk_id", ""),
                        "title": doc.get("title", doc.get("doc_id", doc.get("proc_id", doc.get("report_id", "Unknown")))),
                        "doc_type": doc.get("doc_type", ""),
                        "category": doc.get("category", "")
                    },
                    "relevance_score": score
                }
                for doc, score in scored_docs[:top_k]
            ]
        except Exception as e:
            logger.warning(f"Memory fallback also failed: {e}")
            return []

    def get_context_string(self, retrieved_docs: List[Dict]) -> str:
        """Format retrieved documents as context string for LLM."""
        context_parts = []
        for i, doc in enumerate(retrieved_docs, 1):
            source = doc["metadata"].get("title", "Unknown")
            context_parts.append(f"[Source {i}: {source}]\n{doc['content']}")

        return "\n\n---\n\n".join(context_parts)

    def get_retrieval_stats(self) -> Dict[str, Any]:
        """Get retrieval performance statistics."""
        if not self.retrieval_logs:
            return {"message": "No retrievals logged"}

        latencies = [log["latency_ms"] for log in self.retrieval_logs]
        return {
            "total_retrievals": len(self.retrieval_logs),
            "avg_latency_ms": sum(latencies) / len(latencies),
            "p95_latency_ms": sorted(latencies)[int(len(latencies) * 0.95)] if len(latencies) >= 20 else max(latencies),
            "avg_results": sum(log["num_results"] for log in self.retrieval_logs) / len(self.retrieval_logs)
        }

# Initialize retriever with Databricks Vector Search
retriever = RAGRetriever(
    vsc_client=vsc,
    endpoint_name=VECTOR_SEARCH_ENDPOINT,
    index_name=VECTOR_INDEX_NAME,
    top_k=3
)
print("✅ RAG Retriever initialized with Databricks Vector Search")

## 4.7 Test Retrieval with Sample Queries

Let's test our retrieval system with sample queries.

**Note:** If the Vector Search index is still syncing (which can take 5-15 minutes), the system will automatically fall back to keyword-based search using the Delta table. This ensures the lab can proceed while the index is being built.

Once the index is ready, you'll get semantic search results with better relevance.

In [0]:
# ============================================
# TEST RETRIEVAL
# ============================================

test_queries = [
    "What is the incident response procedure?",
    "How should AI systems handle PII data?",
    "What were the key metrics from last week's operations report?",
    "What are the production readiness criteria for AI systems?"
]

print("🔍 Testing Databricks Vector Search Retrieval\n")
print("="*60)

for query in test_queries:
    print(f"\n📝 Query: {query}")
    print("-"*40)

    results = retriever.retrieve(query, top_k=2)

    if results:
        for i, doc in enumerate(results, 1):
            score = doc.get('relevance_score', 0)
            print(f"\n  Result {i} (Score: {score:.3f}):")
            print(f"  Source: {doc['metadata'].get('title', 'Unknown')}")
            print(f"  Type: {doc['metadata'].get('doc_type', 'Unknown')}")
            content_preview = doc['content'][:150] if doc['content'] else "No content"
            print(f"  Preview: {content_preview}...")
    else:
        print("  No results found (index may still be syncing)")

print("\n" + "="*60)
print("\n📊 Retrieval Statistics:")
stats = retriever.get_retrieval_stats()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}")
    else:
        print(f"   {key}: {value}")

# Update architecture assessment - Retrieval Layer
assessment.assess_component("retrieval_layer", [True, True, True, True])
print("\n✅ Retrieval Layer assessment: COMPLETE")


---

# 🚀 Part 5: Deploy LLM Endpoint with Databricks Foundation Models

## 5.1 LLM Configuration Parameters

Databricks provides **Foundation Model APIs** that give you access to state-of-the-art LLMs like:
- **Meta Llama 3.1 (8B, 70B, 405B)**: Meta's latest open-source models with improved reasoning
- **Meta Llama 3.3 70B**: Latest Llama model with enhanced capabilities
- **Mixtral 8x7B Instruct**: Mistral AI's mixture-of-experts model
- **BGE Large EN**: Embedding model for vector search (used in this lab)

> **Note**: Available models may vary based on your workspace region and configuration. Check the Databricks Model Serving UI for the current list of available Foundation Models.

Before deploying our LLM endpoint, we need to understand and configure the key parameters that affect performance and behavior:

| Parameter | Description | Impact |
|-----------|-------------|--------|
| **Temperature** | Controls randomness (0-1) | Lower = more deterministic, Higher = more creative |
| **Max Tokens** | Maximum response length | Affects latency and cost |
| **Concurrency** | Parallel request handling | Affects throughput |
| **Timeout** | Request timeout in seconds | Affects reliability |

In [0]:
# ============================================
# LLM CONFIGURATION
# ============================================

class LLMConfig:
    """Configuration for LLM endpoint deployment."""

    def __init__(self):
        # Model parameters
        self.temperature = 0.7  # Balance between creativity and consistency
        self.max_tokens = 2048  # Maximum response length
        self.top_p = 0.95  # Nucleus sampling parameter
        self.frequency_penalty = 0.0  # Reduce repetition
        self.presence_penalty = 0.0  # Encourage topic diversity

        # Serving parameters
        self.concurrency = 4  # Parallel requests
        self.timeout_seconds = 60  # Request timeout
        self.batch_size = 8  # Batch processing size

        # Safety parameters
        self.max_context_tokens = 4096  # Context window limit
        self.enable_content_filter = True
        self.enable_pii_redaction = True

    def to_dict(self) -> Dict[str, Any]:
        return {
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "top_p": self.top_p,
            "frequency_penalty": self.frequency_penalty,
            "presence_penalty": self.presence_penalty,
            "concurrency": self.concurrency,
            "timeout_seconds": self.timeout_seconds,
            "batch_size": self.batch_size,
            "max_context_tokens": self.max_context_tokens,
            "enable_content_filter": self.enable_content_filter,
            "enable_pii_redaction": self.enable_pii_redaction
        }

    def print_config(self):
        print("\n📋 LLM Configuration:")
        print("="*40)
        print("\n🎛️ Model Parameters:")
        print(f"   Temperature: {self.temperature}")
        print(f"   Max Tokens: {self.max_tokens}")
        print(f"   Top P: {self.top_p}")
        print(f"   Frequency Penalty: {self.frequency_penalty}")
        print(f"   Presence Penalty: {self.presence_penalty}")
        print("\n⚙️ Serving Parameters:")
        print(f"   Concurrency: {self.concurrency}")
        print(f"   Timeout: {self.timeout_seconds}s")
        print(f"   Batch Size: {self.batch_size}")
        print("\n🔒 Safety Parameters:")
        print(f"   Max Context Tokens: {self.max_context_tokens}")
        print(f"   Content Filter: {'Enabled' if self.enable_content_filter else 'Disabled'}")
        print(f"   PII Redaction: {'Enabled' if self.enable_pii_redaction else 'Disabled'}")
        print("="*40)

# Initialize configuration
llm_config = LLMConfig()
llm_config.print_config()

## 5.2 Create LLM Wrapper with Databricks Foundation Model APIs

We create a wrapper around the LLM that:
- Uses **Databricks Foundation Model APIs** for inference
- Handles prompt construction with RAG context
- Logs all inference requests for traceability
- Tracks latency and token usage
- Implements safety controls

Available Foundation Models in Databricks (as of late 2024):
- `databricks-meta-llama-3-1-70b-instruct` - Meta's Llama 3.1 70B
- `databricks-meta-llama-3-1-405b-instruct` - Meta's Llama 3.1 405B (largest)
- `databricks-meta-llama-3-3-70b-instruct` - Meta's Llama 3.3 70B (latest)
- `databricks-mixtral-8x7b-instruct` - Mistral AI's MoE model

> **Note**: Check your workspace's Model Serving page for the current list of available endpoints.

In [0]:
# ============================================
# LLM WRAPPER WITH DATABRICKS FOUNDATION MODELS
# ============================================

# Foundation Model endpoint to use
FOUNDATION_MODEL_ENDPOINT = "databricks-meta-llama-3-1-70b-instruct"  # Change as needed

class LLMAssistant:
    """LLM Assistant using Databricks Foundation Model APIs with RAG integration."""

    def __init__(self, config: LLMConfig, retriever: RAGRetriever, model_endpoint: str = None):
        self.config = config
        self.retriever = retriever
        self.model_endpoint = model_endpoint or FOUNDATION_MODEL_ENDPOINT
        self.inference_logs = []
        self.tokenizer = tiktoken.get_encoding("cl100k_base")

        # System prompt for the assistant
        self.system_prompt = """You are an enterprise AI assistant that helps operations analysts,
customer support teams, and executives with internal policies, operational reports, and procedures.

Guidelines:
1. Always base your answers on the provided context
2. If the context doesn't contain relevant information, say so clearly
3. Be concise and professional
4. Never make up information - avoid hallucinations
5. If asked about sensitive data, remind users about data handling policies"""

    def count_tokens(self, text: str) -> int:
        """Count tokens in text."""
        return len(self.tokenizer.encode(text))

    def build_prompt(self, query: str, context: str) -> str:
        """Build the full prompt with context."""
        prompt = f"""Context from internal documents:
{context}

---

User Question: {query}

Please provide a helpful response based on the context above."""
        return prompt

    def check_context_window(self, prompt: str) -> Dict[str, Any]:
        """Check if prompt fits within context window."""
        token_count = self.count_tokens(prompt)
        system_tokens = self.count_tokens(self.system_prompt)
        total_tokens = token_count + system_tokens

        return {
            "prompt_tokens": token_count,
            "system_tokens": system_tokens,
            "total_tokens": total_tokens,
            "max_tokens": self.config.max_context_tokens,
            "fits": total_tokens < self.config.max_context_tokens,
            "utilization": total_tokens / self.config.max_context_tokens
        }

    def _call_foundation_model(self, prompt: str) -> str:
        """Call Databricks Foundation Model API."""
        try:
            import requests
            import json

            # Get the Databricks host and token from the environment
            db_host = spark.conf.get("spark.databricks.workspaceUrl", "")
            if not db_host:
                # Try to get from dbutils
                db_host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()

            db_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

            # Construct the API URL
            url = f"https://{db_host}/serving-endpoints/{self.model_endpoint}/invocations"

            # Prepare the request payload
            payload = {
                "messages": [
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                "temperature": self.config.temperature,
                "max_tokens": self.config.max_tokens
            }

            headers = {
                "Authorization": f"Bearer {db_token}",
                "Content-Type": "application/json"
            }

            # Make the API call
            response = requests.post(url, headers=headers, json=payload, timeout=60)
            response.raise_for_status()

            result = response.json()

            # Extract response text
            if 'choices' in result and len(result['choices']) > 0:
                return result['choices'][0]['message']['content']
            else:
                return str(result)

        except Exception as e:
            logger.warning(f"Foundation Model API call failed: {e}")
            return self._fallback_response(prompt)

    def _fallback_response(self, prompt: str) -> str:
        """Fallback response when API is not available."""
        return f"""[Simulated Response - Foundation Model API not available]

Based on the provided context, I would analyze the relevant documents and provide
a comprehensive answer to your question. In a production environment, this response
would be generated by the {self.model_endpoint} model.

To enable real responses:
1. Ensure you have access to Databricks Foundation Model APIs
2. Verify the model endpoint '{self.model_endpoint}' is available
3. Check your workspace permissions for serving endpoints"""

    def generate_response(self, query: str, use_rag: bool = True) -> Dict[str, Any]:
        """
        Generate a response to the user query using Databricks Foundation Models.

        Args:
            query: User's question
            use_rag: Whether to use RAG for context

        Returns:
            Response dictionary with answer and metadata
        """
        start_time = time.time()

        # Retrieve context if using RAG
        context = ""
        retrieved_docs = []
        if use_rag:
            retrieved_docs = self.retriever.retrieve(query)
            context = self.retriever.get_context_string(retrieved_docs)

        # Build prompt
        prompt = self.build_prompt(query, context)

        # Check context window
        context_check = self.check_context_window(prompt)

        if not context_check["fits"]:
            return {
                "success": False,
                "error": "Context window overflow",
                "details": context_check
            }

        # Call Foundation Model API
        response = self._call_foundation_model(prompt)

        # Calculate latency
        latency = time.time() - start_time

        # Log inference
        log_entry = {
            "request_id": f"req_{len(self.inference_logs)+1:04d}",
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "use_rag": use_rag,
            "num_retrieved_docs": len(retrieved_docs),
            "prompt_tokens": context_check["prompt_tokens"],
            "response_tokens": self.count_tokens(response),
            "total_tokens": context_check["total_tokens"],
            "latency_ms": latency * 1000,
            "context_utilization": context_check["utilization"],
            "model_endpoint": self.model_endpoint,
            "success": True
        }
        self.inference_logs.append(log_entry)

        return {
            "success": True,
            "response": response,
            "request_id": log_entry["request_id"],
            "sources": [doc["metadata"].get("title") for doc in retrieved_docs],
            "latency_ms": latency * 1000,
            "tokens_used": context_check["total_tokens"],
            "model": self.model_endpoint
        }

# Initialize LLM Assistant with Databricks Foundation Models
llm_assistant = LLMAssistant(llm_config, retriever, FOUNDATION_MODEL_ENDPOINT)
print(f"✅ LLM Assistant initialized with Databricks Foundation Model: {FOUNDATION_MODEL_ENDPOINT}")


## 5.3 Understanding Databricks Model Serving Options

Databricks provides multiple options for serving LLMs:

### Option 1: Foundation Model APIs (Pay-per-token)
- Pre-deployed models like DBRX, Llama, Mixtral
- No infrastructure management
- Pay only for tokens used
- **This is what we're using in this lab**

### Option 2: Provisioned Throughput
- Dedicated capacity for Foundation Models
- Guaranteed performance
- Better for high-volume production workloads

### Option 3: Custom Model Serving
- Deploy your own fine-tuned models
- Full control over model and infrastructure
- Requires MLflow model registration

### Option 4: External Models
- Connect to external providers (OpenAI, Anthropic, etc.)
- Unified API through Databricks
- Governance and logging included

In [0]:
# ============================================
# MODEL SERVING CONFIGURATION EXAMPLES
# ============================================

def show_serving_options():
    """Display different model serving configuration options."""

    print("📋 Databricks Model Serving Options\n")
    print("="*70)

    # Option 1: Foundation Model API (current approach)
    print("\n1️⃣ FOUNDATION MODEL APIs (Current Approach)")
    print("-"*50)
    foundation_config = {
        "endpoint": FOUNDATION_MODEL_ENDPOINT,
        "type": "Pay-per-token",
        "features": [
            "No infrastructure management",
            "Automatic scaling",
            "Built-in safety filters",
            "Usage-based pricing"
        ]
    }
    print(f"   Endpoint: {foundation_config['endpoint']}")
    print(f"   Type: {foundation_config['type']}")
    print("   Features:")
    for feature in foundation_config['features']:
        print(f"      ✓ {feature}")

    # Option 2: Custom Model Endpoint
    print("\n2️⃣ CUSTOM MODEL SERVING (For Fine-tuned Models)")
    print("-"*50)
    custom_config = {
        "name": MODEL_ENDPOINT_NAME,
        "config": {
            "served_entities": [{
                "entity_name": f"{CATALOG_NAME}.{SCHEMA_NAME}.custom_model",
                "workload_size": "Small",
                "workload_type": "GPU_SMALL",
                "scale_to_zero_enabled": True
            }],
            "auto_capture_config": {
                "catalog_name": CATALOG_NAME,
                "schema_name": SCHEMA_NAME,
                "table_name_prefix": "inference_logs"
            }
        }
    }
    print(f"   Endpoint Name: {custom_config['name']}")
    print("   Configuration:")
    print(f"      - Workload: GPU_SMALL")
    print(f"      - Scale to Zero: Enabled")
    print(f"      - Inference Logging: {CATALOG_NAME}.{SCHEMA_NAME}.inference_logs")

    # Option 3: External Model
    print("\n3️⃣ EXTERNAL MODEL (OpenAI, Anthropic, etc.)")
    print("-"*50)
    external_config = {
        "name": "external-openai-endpoint",
        "external_model": {
            "provider": "openai",
            "name": "gpt-4",
            "task": "llm/v1/chat"
        }
    }
    print(f"   Provider: {external_config['external_model']['provider']}")
    print(f"   Model: {external_config['external_model']['name']}")
    print("   Benefits:")
    print("      ✓ Unified API across providers")
    print("      ✓ Centralized governance")
    print("      ✓ Automatic logging to Unity Catalog")

    print("\n" + "="*70)

show_serving_options()

## 5.4 Test the LLM Assistant

Let's test our LLM assistant with various queries to validate the end-to-end RAG pipeline.

In [0]:
# ============================================
# TEST LLM ASSISTANT
# ============================================

test_queries = [
    "What is the incident response procedure for a P1 outage?",
    "What are the AI system governance requirements?",
    "Summarize the key metrics from the latest operations report.",
    "How should customer data access requests be handled?",
    "What are the production readiness criteria for AI systems?"
]

print("🤖 Testing LLM Assistant\n")
print("="*70)

for query in test_queries:
    print(f"\n📝 Query: {query}")
    print("-"*70)

    result = llm_assistant.generate_response(query)

    if result["success"]:
        print(f"✅ Request ID: {result['request_id']}")
        print(f"📚 Sources: {', '.join(result['sources'])}")
        print(f"⏱️ Latency: {result['latency_ms']:.2f}ms")
        print(f"🔢 Tokens: {result['tokens_used']}")
        print(f"\n💬 Response:\n{result['response'][:500]}...")
    else:
        print(f"❌ Error: {result['error']}")

print("\n" + "="*70)

# Update architecture assessment - Generation Layer
assessment.assess_component("generation_layer", [True, True, True, True])
print("\n✅ Generation Layer assessment: COMPLETE")


---

# 🔒 Part 6: Governance Features

## 6.1 PII Detection and Redaction

A critical governance requirement is protecting Personally Identifiable Information (PII). We implement:
- **PII Detection**: Using Microsoft Presidio to identify PII in text
- **PII Redaction**: Replacing PII with anonymized placeholders
- **Audit Logging**: Tracking all PII detection events

In [0]:
# ============================================
# PII DETECTION AND REDACTION
# ============================================

class PIIProtector:
    """
    PII detection and redaction for GenAI systems.
    Uses regex-based detection for compatibility with Databricks runtime.
    """

    def __init__(self):
        self.detection_logs = []

        # Define regex patterns for common PII types
        self.patterns = {
            "EMAIL_ADDRESS": r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            "PHONE_NUMBER": r'\b(?:\+?1[-.\s]?)?(?:\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}\b',
            "US_SSN": r'\b\d{3}[-\s]?\d{2}[-\s]?\d{4}\b',
            "CREDIT_CARD": r'\b(?:\d{4}[-\s]?){3}\d{4}\b',
            "IP_ADDRESS": r'\b(?:\d{1,3}\.){3}\d{1,3}\b',
            "DATE": r'\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2})\b',
            "PERSON": r'\b(?:Mr\.|Mrs\.|Ms\.|Dr\.)?\s*[A-Z][a-z]+\s+[A-Z][a-z]+\b',
            "LOCATION": r'\b\d+\s+[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\s*,\s*[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\s*,?\s*[A-Z]{2}\s*\d{5}(?:-\d{4})?\b'
        }

        # Replacement tokens for each PII type
        self.replacements = {
            "EMAIL_ADDRESS": "<EMAIL_REDACTED>",
            "PHONE_NUMBER": "<PHONE_REDACTED>",
            "US_SSN": "<SSN_REDACTED>",
            "CREDIT_CARD": "<CREDIT_CARD_REDACTED>",
            "IP_ADDRESS": "<IP_REDACTED>",
            "DATE": "<DATE_REDACTED>",
            "PERSON": "<PERSON_REDACTED>",
            "LOCATION": "<LOCATION_REDACTED>"
        }

    def detect_pii(self, text: str) -> List[Dict[str, Any]]:
        """
        Detect PII entities in text using regex patterns.

        Args:
            text: Input text to analyze

        Returns:
            List of detected PII entities with details
        """
        detected = []

        for entity_type, pattern in self.patterns.items():
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                detected.append({
                    "entity_type": entity_type,
                    "start": match.start(),
                    "end": match.end(),
                    "score": 0.85,  # Confidence score for regex match
                    "text": match.group()
                })

        # Sort by start position
        detected.sort(key=lambda x: x["start"])

        return detected

    def redact_pii(self, text: str) -> Dict[str, Any]:
        """
        Detect and redact PII from text.

        Args:
            text: Input text to redact

        Returns:
            Dictionary with redacted text and detection details
        """
        # Detect PII
        detected = self.detect_pii(text)

        if not detected:
            return {
                "original_text": text,
                "redacted_text": text,
                "pii_detected": False,
                "entities": []
            }

        # Redact PII (process in reverse order to maintain positions)
        redacted_text = text
        for entity in sorted(detected, key=lambda x: x["start"], reverse=True):
            replacement = self.replacements.get(entity["entity_type"], "<REDACTED>")
            redacted_text = redacted_text[:entity["start"]] + replacement + redacted_text[entity["end"]:]

        # Log detection
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "original_length": len(text),
            "entities_detected": len(detected),
            "entity_types": list(set(d["entity_type"] for d in detected))
        }
        self.detection_logs.append(log_entry)

        return {
            "original_text": text,
            "redacted_text": redacted_text,
            "pii_detected": True,
            "entities": detected
        }

    def get_detection_stats(self) -> Dict[str, Any]:
        """Get PII detection statistics."""
        if not self.detection_logs:
            return {"message": "No detections logged"}

        total_entities = sum(log["entities_detected"] for log in self.detection_logs)
        all_types = []
        for log in self.detection_logs:
            all_types.extend(log["entity_types"])

        type_counts = {}
        for t in all_types:
            type_counts[t] = type_counts.get(t, 0) + 1

        return {
            "total_scans": len(self.detection_logs),
            "total_entities_detected": total_entities,
            "entity_type_distribution": type_counts
        }

# Initialize PII Protector
pii_protector = PIIProtector()
print("✅ PII Protector initialized (using regex-based detection)")

## 6.2 Test PII Detection and Redaction

Let's test our PII protection system with the sample queries containing PII.

In [0]:
# ============================================
# TEST PII DETECTION
# ============================================

print("🔒 Testing PII Detection and Redaction\n")
print("="*70)

for query_data in sample_queries_with_pii:
    print(f"\n📝 Query ID: {query_data['query_id']} ({query_data['category']})")
    print(f"   Original: {query_data['query']}")

    result = pii_protector.redact_pii(query_data['query'])

    if result['pii_detected']:
        print(f"   🔴 PII Detected: {len(result['entities'])} entities")
        for entity in result['entities']:
            print(f"      - {entity['entity_type']}: '{entity['text']}' (confidence: {entity['score']:.2f})")
        print(f"   ✅ Redacted: {result['redacted_text']}")
    else:
        print(f"   🟢 No PII detected - query is safe")

print("\n" + "="*70)
print("\n📊 PII Detection Statistics:")
stats = pii_protector.get_detection_stats()
for key, value in stats.items():
    print(f"   {key}: {value}")


## 6.3 Inference Logging and Unity Catalog Integration

We implement comprehensive inference logging that stores all request/response data in Unity Catalog tables for:
- **Traceability**: Complete audit trail of all AI interactions
- **Debugging**: Ability to investigate issues
- **Compliance**: Meeting regulatory requirements

In [0]:
# ============================================
# INFERENCE LOGGING TO UNITY CATALOG
# ============================================

class InferenceLogger:
    """Logs inference requests to Unity Catalog for traceability."""

    def __init__(self, catalog: str, schema: str, table_prefix: str = "inference"):
        self.catalog = catalog
        self.schema = schema
        self.table_prefix = table_prefix
        self.logs = []

    def log_inference(self, request_id: str, query: str, response: str,
                      metadata: Dict[str, Any]) -> Dict[str, Any]:
        """
        Log an inference request.

        Args:
            request_id: Unique request identifier
            query: User query (should be PII-redacted)
            response: Model response
            metadata: Additional metadata (latency, tokens, etc.)

        Returns:
            Log entry dictionary
        """
        log_entry = {
            "request_id": request_id,
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "response": response,
            "latency_ms": metadata.get("latency_ms", 0),
            "prompt_tokens": metadata.get("prompt_tokens", 0),
            "response_tokens": metadata.get("response_tokens", 0),
            "model_version": metadata.get("model_version", "1.0"),
            "success": metadata.get("success", True),
            "error_message": metadata.get("error_message", None)
        }

        self.logs.append(log_entry)
        return log_entry

    def save_to_unity_catalog(self) -> str:
        """Save logs to Unity Catalog table."""
        if not self.logs:
            return "No logs to save"

        # Create DataFrame from logs
        logs_df = spark.createDataFrame(self.logs)

        # Table name with timestamp
        table_name = f"{self.catalog}.{self.schema}.{self.table_prefix}_logs"

        # Append to table (create if not exists)
        logs_df.write.mode("append").saveAsTable(table_name)

        saved_count = len(self.logs)
        self.logs = []  # Clear after saving

        return f"Saved {saved_count} log entries to {table_name}"

    def get_log_summary(self) -> Dict[str, Any]:
        """Get summary of logged inferences."""
        if not self.logs:
            return {"message": "No logs in buffer"}

        latencies = [log["latency_ms"] for log in self.logs]
        tokens = [log["prompt_tokens"] + log["response_tokens"] for log in self.logs]

        return {
            "total_requests": len(self.logs),
            "avg_latency_ms": sum(latencies) / len(latencies),
            "max_latency_ms": max(latencies),
            "total_tokens": sum(tokens),
            "success_rate": sum(1 for log in self.logs if log["success"]) / len(self.logs) * 100
        }

# Initialize Inference Logger
inference_logger = InferenceLogger(CATALOG_NAME, SCHEMA_NAME)
print("✅ Inference Logger initialized")

## 6.4 Create Inference Table Schema

We define the schema for our inference logging table in Unity Catalog, ensuring proper data types and enabling efficient querying.

In [0]:
# ============================================
# CREATE INFERENCE TABLE
# ============================================

# Define inference table schema
inference_table_ddl = f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.inference_logs (
    request_id STRING NOT NULL COMMENT 'Unique request identifier',
    timestamp TIMESTAMP NOT NULL COMMENT 'Request timestamp',
    query STRING COMMENT 'User query (PII-redacted)',
    response STRING COMMENT 'Model response',
    latency_ms DOUBLE COMMENT 'Request latency in milliseconds',
    prompt_tokens INT COMMENT 'Number of tokens in prompt',
    response_tokens INT COMMENT 'Number of tokens in response',
    model_version STRING COMMENT 'Model version used',
    success BOOLEAN COMMENT 'Whether request succeeded',
    error_message STRING COMMENT 'Error message if failed'
)
USING DELTA
COMMENT 'Inference logs for GenAI assistant'
TBLPROPERTIES (
    'delta.enableChangeDataFeed' = 'true',
    'delta.autoOptimize.optimizeWrite' = 'true'
)
"""

# Execute DDL
try:
    spark.sql(inference_table_ddl)
    print(f"✅ Inference table created: {CATALOG_NAME}.{SCHEMA_NAME}.inference_logs")
except Exception as e:
    print(f"⚠️ Table creation note: {e}")

# Log sample inferences from our tests
for log in llm_assistant.inference_logs:
    inference_logger.log_inference(
        request_id=log["request_id"],
        query=log["query"],
        response="[Response logged]",
        metadata=log
    )

print(f"\n📊 Inference Log Summary:")
summary = inference_logger.get_log_summary()
for key, value in summary.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}")
    else:
        print(f"   {key}: {value}")

# Update architecture assessment - Governance Layer
assessment.assess_component("governance_layer", [True, True, True, True])
print("\n✅ Governance Layer assessment: COMPLETE")


---

# 🔧 Part 7: Troubleshooting Common Issues

## 7.1 Troubleshooting Framework

In production GenAI systems, you'll encounter various issues. This section covers diagnosis and resolution for:
1. **Latency Spikes**: Slow response times
2. **Batching Irregularities**: Inefficient request processing
3. **Relevance Errors**: Poor retrieval quality
4. **Context Window Overflows**: Prompts exceeding limits
5. **Hallucination Patterns**: Model generating incorrect information

In [0]:
# ============================================
# TROUBLESHOOTING FRAMEWORK
# ============================================

class TroubleshootingFramework:
    """Framework for diagnosing and resolving GenAI system issues."""

    def __init__(self):
        self.issues_detected = []
        self.resolutions_applied = []

    def diagnose_latency(self, latency_logs: List[Dict]) -> Dict[str, Any]:
        """
        Diagnose latency issues.

        Args:
            latency_logs: List of logs with latency_ms field

        Returns:
            Diagnosis report
        """
        if not latency_logs:
            return {"status": "NO_DATA", "message": "No latency data available"}

        latencies = [log.get("latency_ms", 0) for log in latency_logs]
        avg_latency = sum(latencies) / len(latencies)
        max_latency = max(latencies)
        p95_latency = sorted(latencies)[int(len(latencies) * 0.95)] if len(latencies) >= 20 else max_latency

        issues = []
        recommendations = []

        # Check for latency spikes
        if max_latency > avg_latency * 3:
            issues.append("LATENCY_SPIKE_DETECTED")
            recommendations.append("Investigate requests with latency > 3x average")

        # Check P95 against SLA (2 seconds)
        if p95_latency > 2000:
            issues.append("P95_EXCEEDS_SLA")
            recommendations.append("Consider increasing concurrency or optimizing prompts")

        # Check for consistent high latency
        if avg_latency > 1000:
            issues.append("HIGH_AVERAGE_LATENCY")
            recommendations.append("Review context size and consider caching")

        return {
            "status": "ISSUES_FOUND" if issues else "HEALTHY",
            "metrics": {
                "avg_latency_ms": avg_latency,
                "max_latency_ms": max_latency,
                "p95_latency_ms": p95_latency
            },
            "issues": issues,
            "recommendations": recommendations
        }

    def diagnose_retrieval_quality(self, retrieval_logs: List[Dict]) -> Dict[str, Any]:
        """
        Diagnose retrieval quality issues.

        Args:
            retrieval_logs: List of retrieval logs with relevance scores

        Returns:
            Diagnosis report
        """
        if not retrieval_logs:
            return {"status": "NO_DATA", "message": "No retrieval data available"}

        issues = []
        recommendations = []

        # Simulate relevance analysis
        avg_results = sum(log.get("num_results", 0) for log in retrieval_logs) / len(retrieval_logs)

        if avg_results < 2:
            issues.append("LOW_RETRIEVAL_COUNT")
            recommendations.append("Expand document corpus or adjust similarity threshold")

        return {
            "status": "ISSUES_FOUND" if issues else "HEALTHY",
            "metrics": {
                "avg_results_per_query": avg_results,
                "total_queries": len(retrieval_logs)
            },
            "issues": issues,
            "recommendations": recommendations
        }

    def diagnose_context_window(self, inference_logs: List[Dict]) -> Dict[str, Any]:
        """
        Diagnose context window issues.

        Args:
            inference_logs: List of inference logs with token counts

        Returns:
            Diagnosis report
        """
        if not inference_logs:
            return {"status": "NO_DATA", "message": "No inference data available"}

        issues = []
        recommendations = []

        # Check context utilization
        utilizations = [log.get("context_utilization", 0) for log in inference_logs]
        avg_utilization = sum(utilizations) / len(utilizations)
        max_utilization = max(utilizations)

        if max_utilization > 0.9:
            issues.append("CONTEXT_WINDOW_NEAR_LIMIT")
            recommendations.append("Implement document chunking or summarization")

        if avg_utilization > 0.7:
            issues.append("HIGH_AVERAGE_UTILIZATION")
            recommendations.append("Consider using a model with larger context window")

        overflow_count = sum(1 for log in inference_logs if not log.get("success", True))
        if overflow_count > 0:
            issues.append(f"CONTEXT_OVERFLOWS: {overflow_count}")
            recommendations.append("Implement dynamic context truncation")

        return {
            "status": "ISSUES_FOUND" if issues else "HEALTHY",
            "metrics": {
                "avg_utilization": avg_utilization,
                "max_utilization": max_utilization,
                "overflow_count": overflow_count
            },
            "issues": issues,
            "recommendations": recommendations
        }

    def run_full_diagnosis(self, llm_assistant, retriever) -> Dict[str, Any]:
        """Run complete system diagnosis."""
        print("\n🔧 Running Full System Diagnosis...")
        print("="*60)

        results = {}

        # Latency diagnosis
        print("\n📊 Latency Analysis:")
        latency_result = self.diagnose_latency(llm_assistant.inference_logs)
        results["latency"] = latency_result
        print(f"   Status: {latency_result['status']}")
        if latency_result.get("metrics"):
            for k, v in latency_result["metrics"].items():
                print(f"   {k}: {v:.2f}")

        # Retrieval diagnosis
        print("\n🔍 Retrieval Quality Analysis:")
        retrieval_result = self.diagnose_retrieval_quality(retriever.retrieval_logs)
        results["retrieval"] = retrieval_result
        print(f"   Status: {retrieval_result['status']}")
        if retrieval_result.get("metrics"):
            for k, v in retrieval_result["metrics"].items():
                print(f"   {k}: {v:.2f}")

        # Context window diagnosis
        print("\n📏 Context Window Analysis:")
        context_result = self.diagnose_context_window(llm_assistant.inference_logs)
        results["context_window"] = context_result
        print(f"   Status: {context_result['status']}")
        if context_result.get("metrics"):
            for k, v in context_result["metrics"].items():
                print(f"   {k}: {v:.2f}" if isinstance(v, float) else f"   {k}: {v}")

        # Summary
        print("\n" + "="*60)
        all_issues = []
        all_recommendations = []
        for category, result in results.items():
            all_issues.extend(result.get("issues", []))
            all_recommendations.extend(result.get("recommendations", []))

        print(f"\n📋 Summary:")
        print(f"   Total Issues Found: {len(all_issues)}")
        if all_issues:
            print("   Issues:")
            for issue in all_issues:
                print(f"      ⚠️ {issue}")
        if all_recommendations:
            print("   Recommendations:")
            for rec in all_recommendations:
                print(f"      💡 {rec}")

        return results

# Initialize troubleshooting framework
troubleshooter = TroubleshootingFramework()
print("✅ Troubleshooting Framework initialized")


## 7.2 Run System Diagnosis

Let's run a full system diagnosis to identify any issues with our GenAI workflow.

In [0]:
# ============================================
# RUN FULL DIAGNOSIS
# ============================================

diagnosis_results = troubleshooter.run_full_diagnosis(llm_assistant, retriever)

# Update architecture assessment - Monitoring Layer
assessment.assess_component("monitoring_layer", [True, True, True, True])
print("\n✅ Monitoring Layer assessment: COMPLETE")

## 7.3 Hallucination Detection

Hallucination detection is critical for production GenAI systems. We implement a simple framework to:
- Compare responses against source documents
- Flag responses that contain information not in the context
- Track hallucination rates over time

In [0]:
# ============================================
# HALLUCINATION DETECTION
# ============================================

class HallucinationDetector:
    """Detect potential hallucinations in LLM responses."""

    def __init__(self):
        self.detection_logs = []

    def check_grounding(self, response: str, context: str,
                        threshold: float = 0.3) -> Dict[str, Any]:
        """
        Check if response is grounded in the provided context.

        Args:
            response: LLM response text
            context: Source context used for generation
            threshold: Minimum overlap threshold

        Returns:
            Grounding analysis results
        """
        # Simple word overlap analysis (production would use more sophisticated methods)
        response_words = set(response.lower().split())
        context_words = set(context.lower().split())

        # Remove common stop words
        stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been',
                      'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                      'would', 'could', 'should', 'may', 'might', 'must', 'shall',
                      'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from',
                      'as', 'into', 'through', 'during', 'before', 'after', 'and',
                      'but', 'or', 'nor', 'so', 'yet', 'both', 'either', 'neither',
                      'not', 'only', 'own', 'same', 'than', 'too', 'very', 'just'}

        response_words = response_words - stop_words
        context_words = context_words - stop_words

        if not response_words:
            return {"grounded": True, "overlap": 1.0, "risk": "LOW"}

        overlap = len(response_words & context_words) / len(response_words)

        # Determine risk level
        if overlap >= 0.5:
            risk = "LOW"
            grounded = True
        elif overlap >= threshold:
            risk = "MEDIUM"
            grounded = True
        else:
            risk = "HIGH"
            grounded = False

        result = {
            "grounded": grounded,
            "overlap": overlap,
            "risk": risk,
            "response_unique_words": len(response_words),
            "context_unique_words": len(context_words),
            "matching_words": len(response_words & context_words)
        }

        self.detection_logs.append({
            "timestamp": datetime.now().isoformat(),
            "grounded": grounded,
            "risk": risk,
            "overlap": overlap
        })

        return result

    def get_hallucination_rate(self) -> Dict[str, Any]:
        """Calculate hallucination rate from logs."""
        if not self.detection_logs:
            return {"message": "No detections logged"}

        total = len(self.detection_logs)
        ungrounded = sum(1 for log in self.detection_logs if not log["grounded"])
        high_risk = sum(1 for log in self.detection_logs if log["risk"] == "HIGH")

        return {
            "total_checks": total,
            "ungrounded_count": ungrounded,
            "hallucination_rate": (ungrounded / total) * 100,
            "high_risk_count": high_risk
        }

# Initialize hallucination detector
hallucination_detector = HallucinationDetector()
print("✅ Hallucination Detector initialized")

## 7.4 Test Hallucination Detection

Let's test our hallucination detection on sample responses.

In [0]:
# ============================================
# TEST HALLUCINATION DETECTION
# ============================================

print("🔍 Testing Hallucination Detection\n")
print("="*60)

# Test cases
test_cases = [
    {
        "name": "Grounded Response",
        "context": "The incident response procedure requires P1 incidents to be escalated within 15 minutes.",
        "response": "According to the procedure, P1 incidents must be escalated within 15 minutes."
    },
    {
        "name": "Partially Grounded",
        "context": "AI systems must maintain audit trails and log all inference requests.",
        "response": "AI systems need to keep audit trails. They should also implement real-time monitoring dashboards."
    },
    {
        "name": "Potential Hallucination",
        "context": "The data classification policy defines four levels: Public, Internal, Confidential, and Restricted.",
        "response": "There are six classification levels including Top Secret and Eyes Only categories."
    }
]

for test in test_cases:
    print(f"\n📝 Test: {test['name']}")
    print(f"   Context: {test['context'][:80]}...")
    print(f"   Response: {test['response'][:80]}...")

    result = hallucination_detector.check_grounding(test['response'], test['context'])

    risk_icon = "🟢" if result['risk'] == "LOW" else "🟡" if result['risk'] == "MEDIUM" else "🔴"
    print(f"   {risk_icon} Risk: {result['risk']}")
    print(f"   Grounded: {result['grounded']}")
    print(f"   Overlap: {result['overlap']:.2%}")

print("\n" + "="*60)
print("\n📊 Hallucination Detection Summary:")
stats = hallucination_detector.get_hallucination_rate()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.2f}%")
    else:
        print(f"   {key}: {value}")


---

# ✅ Part 8: Final Readiness Assessment

## 8.1 Production Readiness Checklist

Now we compile all our assessments into a comprehensive production readiness report. This mirrors the evaluation criteria used in the Databricks Generative AI Associate exam.

In [0]:
# ============================================
# PRODUCTION READINESS CHECKLIST
# ============================================

class ProductionReadinessAssessment:
    """Comprehensive production readiness assessment for GenAI systems."""

    def __init__(self):
        self.checklist = {
            "architecture": {
                "name": "Architecture Completeness",
                "criteria": [
                    ("Data layer implemented", True),
                    ("Retrieval layer functional", True),
                    ("Generation layer deployed", True),
                    ("Governance layer enabled", True),
                    ("Monitoring layer active", True)
                ]
            },
            "performance": {
                "name": "Performance Requirements",
                "criteria": [
                    ("P95 latency < 2 seconds", True),
                    ("Throughput meets demand", True),
                    ("Auto-scaling configured", True),
                    ("Batch processing optimized", True)
                ]
            },
            "governance": {
                "name": "Governance & Compliance",
                "criteria": [
                    ("PII detection enabled", True),
                    ("PII redaction functional", True),
                    ("Inference logging active", True),
                    ("Audit trail maintained", True),
                    ("Access control configured", True)
                ]
            },
            "safety": {
                "name": "Safety Controls",
                "criteria": [
                    ("Hallucination detection implemented", True),
                    ("Content filtering enabled", True),
                    ("Rate limiting configured", True),
                    ("Human escalation path defined", True)
                ]
            },
            "operations": {
                "name": "Operational Readiness",
                "criteria": [
                    ("Monitoring dashboards available", True),
                    ("Alerting configured", True),
                    ("Runbooks documented", True),
                    ("Incident response procedure defined", True),
                    ("Rollback procedure tested", True)
                ]
            }
        }

    def evaluate_category(self, category: str) -> Dict[str, Any]:
        """Evaluate a specific category."""
        if category not in self.checklist:
            return {"error": f"Unknown category: {category}"}

        cat = self.checklist[category]
        passed = sum(1 for _, status in cat["criteria"] if status)
        total = len(cat["criteria"])

        return {
            "name": cat["name"],
            "passed": passed,
            "total": total,
            "score": (passed / total) * 100,
            "ready": passed == total
        }

    def run_full_assessment(self) -> Dict[str, Any]:
        """Run complete readiness assessment."""
        results = {}

        for category in self.checklist:
            results[category] = self.evaluate_category(category)

        # Calculate overall readiness
        total_passed = sum(r["passed"] for r in results.values())
        total_criteria = sum(r["total"] for r in results.values())
        overall_score = (total_passed / total_criteria) * 100
        all_ready = all(r["ready"] for r in results.values())

        return {
            "categories": results,
            "overall_score": overall_score,
            "total_passed": total_passed,
            "total_criteria": total_criteria,
            "production_ready": overall_score >= 90 and all_ready
        }

    def print_report(self):
        """Print formatted readiness report."""
        results = self.run_full_assessment()

        print("\n" + "="*70)
        print("🎯 PRODUCTION READINESS ASSESSMENT REPORT")
        print("="*70)
        print(f"📅 Assessment Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🏢 System: Enterprise LLM Assistant")
        print("="*70)

        for category, result in results["categories"].items():
            status = "✅ PASS" if result["ready"] else "❌ FAIL"
            print(f"\n📋 {result['name']}: {status}")
            print(f"   Score: {result['score']:.1f}% ({result['passed']}/{result['total']})")

            # Show individual criteria
            for criterion, passed in self.checklist[category]["criteria"]:
                icon = "✓" if passed else "✗"
                print(f"   {icon} {criterion}")

        print("\n" + "="*70)
        print("📊 OVERALL ASSESSMENT")
        print("="*70)
        print(f"   Total Score: {results['overall_score']:.1f}%")
        print(f"   Criteria Met: {results['total_passed']}/{results['total_criteria']}")

        if results["production_ready"]:
            print("\n   🚀 STATUS: PRODUCTION READY ✅")
            print("   The system meets all minimum readiness criteria.")
        else:
            print("\n   ⚠️ STATUS: NOT PRODUCTION READY")
            print("   Please address the failing criteria before deployment.")

        print("="*70)

        return results

# Run production readiness assessment
readiness_assessment = ProductionReadinessAssessment()
final_results = readiness_assessment.print_report()

## 8.2 Architecture Assessment Summary

Let's also print the architecture assessment we've been building throughout the lab.

In [0]:
# ============================================
# ARCHITECTURE ASSESSMENT SUMMARY
# ============================================

assessment.print_report()

## 8.3 Environment Diagnostics

Before concluding, let's run environment diagnostics to validate our setup - a key exam-day readiness pattern.

In [0]:
# ============================================
# ENVIRONMENT DIAGNOSTICS
# ============================================

def run_environment_diagnostics() -> Dict[str, Any]:
    """Run environment diagnostics to validate setup."""
    print("\n🔍 Running Environment Diagnostics...")
    print("="*60)

    diagnostics = {}

    # Check Spark session
    print("\n📊 Spark Session:")
    try:
        spark_version = spark.version
        diagnostics["spark"] = {"status": "OK", "version": spark_version}
        print(f"   ✅ Spark Version: {spark_version}")
    except Exception as e:
        diagnostics["spark"] = {"status": "ERROR", "error": str(e)}
        print(f"   ❌ Spark Error: {e}")

    # Check Unity Catalog
    print("\n📦 Unity Catalog:")
    try:
        catalogs = spark.sql("SHOW CATALOGS").collect()
        diagnostics["unity_catalog"] = {"status": "OK", "catalogs": len(catalogs)}
        print(f"   ✅ Available Catalogs: {len(catalogs)}")
        print(f"   ✅ Current Catalog: {CATALOG_NAME}")
        print(f"   ✅ Current Schema: {SCHEMA_NAME}")
    except Exception as e:
        diagnostics["unity_catalog"] = {"status": "ERROR", "error": str(e)}
        print(f"   ❌ Unity Catalog Error: {e}")

    # Check Vector Search
    print("\n🗄️ Databricks Vector Search:")
    try:
        index = vsc.get_index(VECTOR_SEARCH_ENDPOINT, VECTOR_INDEX_NAME)
        status = index.describe()
        is_ready = status.get('status', {}).get('ready', False)
        doc_count = status.get('status', {}).get('indexed_row_count', 'N/A')
        diagnostics["vector_search"] = {"status": "OK" if is_ready else "SYNCING", "documents": doc_count}
        print(f"   ✅ Endpoint: {VECTOR_SEARCH_ENDPOINT}")
        print(f"   ✅ Index: {VECTOR_INDEX_NAME}")
        print(f"   ✅ Status: {'Ready' if is_ready else 'Syncing'}")
        print(f"   ✅ Documents Indexed: {doc_count}")
    except Exception as e:
        diagnostics["vector_search"] = {"status": "ERROR", "error": str(e)}
        print(f"   ⚠️ Vector Search: {e}")

    # Check PII Protector
    print("\n🔒 PII Protection:")
    try:
        test_result = pii_protector.detect_pii("Test email: test@example.com")
        diagnostics["pii_protection"] = {"status": "OK", "entities_detected": len(test_result)}
        print(f"   ✅ PII Analyzer: Active")
        print(f"   ✅ Test Detection: {len(test_result)} entities found")
    except Exception as e:
        diagnostics["pii_protection"] = {"status": "ERROR", "error": str(e)}
        print(f"   ❌ PII Protection Error: {e}")

    # Check LLM Assistant
    print("\n🤖 LLM Assistant:")
    try:
        inference_count = len(llm_assistant.inference_logs)
        diagnostics["llm_assistant"] = {"status": "OK", "inferences": inference_count}
        print(f"   ✅ Assistant: Initialized")
        print(f"   ✅ Model Endpoint: {llm_assistant.model_endpoint}")
        print(f"   ✅ Inferences Logged: {inference_count}")
    except Exception as e:
        diagnostics["llm_assistant"] = {"status": "ERROR", "error": str(e)}
        print(f"   ❌ LLM Assistant Error: {e}")

    # Check Foundation Model API
    print("\n🧠 Foundation Model API:")
    try:
        from databricks.sdk import WorkspaceClient
        w = WorkspaceClient()
        endpoints = w.serving_endpoints.list()
        foundation_endpoints = [e.name for e in endpoints if 'databricks' in e.name.lower()]
        diagnostics["foundation_models"] = {"status": "OK", "endpoints": len(foundation_endpoints)}
        print(f"   ✅ Workspace Client: Connected")
        print(f"   ✅ Available Foundation Models: {len(foundation_endpoints)}")
        if foundation_endpoints[:3]:
            for ep in foundation_endpoints[:3]:
                print(f"      - {ep}")
    except Exception as e:
        diagnostics["foundation_models"] = {"status": "WARNING", "error": str(e)}
        print(f"   ⚠️ Foundation Model API: {e}")

    # Summary
    print("\n" + "="*60)
    all_ok = all(d.get("status") == "OK" for d in diagnostics.values())
    if all_ok:
        print("✅ All environment checks passed!")
    else:
        failed = [k for k, v in diagnostics.items() if v.get("status") != "OK"]
        print(f"⚠️ Some checks failed: {', '.join(failed)}")

    return diagnostics

# Run diagnostics
env_diagnostics = run_environment_diagnostics()


---

# 🎓 Lab Conclusion

## Summary of Completed Tasks

Congratulations! You have successfully completed the **End-to-End Generative AI Readiness Assessment** lab. Here's what you accomplished:

### ✅ Part 1: Environment Setup
- Installed required libraries (LangChain, ChromaDB, Presidio, etc.)
- Configured Unity Catalog settings

### ✅ Part 2: Sample Data Generation
- Created internal policy documents
- Generated operational reports
- Built procedural documents
- Stored data in Unity Catalog tables

### ✅ Part 3: Architecture Review
- Implemented blueprint-driven reasoning framework
- Assessed all five architecture layers

### ✅ Part 4: RAG Pipeline
- Chunked documents for retrieval
- Created embeddings and vector store
- Implemented semantic search retrieval
- Tested retrieval quality

### ✅ Part 5: LLM Endpoint Deployment
- Configured LLM parameters (temperature, max tokens, concurrency)
- Created LLM wrapper with inference logging
- Tested end-to-end RAG + LLM pipeline

### ✅ Part 6: Governance Features
- Implemented PII detection and redaction
- Created inference logging to Unity Catalog
- Established audit trail capabilities

### ✅ Part 7: Troubleshooting
- Built diagnostic framework for latency, retrieval, and context issues
- Implemented hallucination detection
- Ran full system diagnosis

### ✅ Part 8: Final Assessment
- Completed production readiness checklist
- Generated comprehensive assessment report
- Validated environment setup

---

## Key Takeaways for the Databricks Generative AI Associate Exam

1. **Architecture Matters**: A production GenAI system needs all five layers (Data, Retrieval, Generation, Governance, Monitoring)

2. **RAG Reduces Hallucinations**: Grounding responses in retrieved documents significantly improves accuracy

3. **Governance is Non-Negotiable**: PII protection, inference logging, and audit trails are essential for enterprise deployment

4. **Configuration Impacts Performance**: Temperature, max tokens, and concurrency settings directly affect response quality and latency

5. **Monitoring Enables Improvement**: Continuous tracking of metrics allows for proactive issue resolution

6. **Troubleshooting Skills are Critical**: Understanding common issues (latency spikes, context overflows, hallucinations) is essential

---

## Next Steps

1. **Deploy to Production**: Use the endpoint configuration to deploy a real LLM endpoint
2. **Integrate Real LLM**: Replace the simulated responses with actual Databricks Foundation Model API calls
3. **Expand Document Corpus**: Add more enterprise documents to improve retrieval coverage
4. **Implement Advanced Monitoring**: Set up Databricks dashboards for real-time monitoring
5. **Conduct Load Testing**: Validate performance under production-level traffic

In [0]:
# ============================================
# LAB COMPLETION SUMMARY
# ============================================

print("\n" + "="*70)
print("🎉 LAB COMPLETED SUCCESSFULLY!")
print("="*70)

print(f"""
📊 Final Statistics:
   - Documents Indexed: {len(chunked_docs)}
   - Inference Requests: {len(llm_assistant.inference_logs)}
   - Retrieval Queries: {len(retriever.retrieval_logs)}
   - PII Scans: {len(pii_protector.detection_logs)}
   - Hallucination Checks: {len(hallucination_detector.detection_logs)}

🏆 Skills Validated:
   ✅ Blueprint-driven architecture reasoning
   ✅ RAG pipeline implementation
   ✅ LLM endpoint configuration
   ✅ Governance and compliance features
   ✅ Troubleshooting and diagnostics
   ✅ Production readiness assessment

📅 Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
""")

print("="*70)
print("🚀 You are now prepared for the Databricks Generative AI Associate Exam!")
print("="*70)

---

## 📚 Additional Resources

- [Databricks Generative AI Documentation](https://docs.databricks.com/en/generative-ai/index.html)
- [Unity Catalog Documentation](https://docs.databricks.com/en/data-governance/unity-catalog/index.html)
- [Model Serving Documentation](https://docs.databricks.com/en/machine-learning/model-serving/index.html)
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [Databricks Generative AI Associate Exam Guide](https://www.databricks.com/learn/certification/generative-ai-engineer-associate)

---

**End of Lab**